# Initialize docker for google colab

In [ ]:
%%bash
sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg
install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
chmod a+r /etc/apt/keyrings/docker.gpg
echo \
  "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
  "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,079 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [81.0 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,482 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 http://security.ubuntu.com/ub

In [ ]:
%%bash
sudo apt update
sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Get:2 https://download.docker.com/linux/ubuntu focal InRelease [57.7 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 https://download.docker.com/linux/ubuntu focal/stable amd64 Packages [35.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 93.4 kB in 2s (55.7 kB/s)
Reading package lists...
Building d



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 15.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [ ]:
%%script bash --bg --out docker_out --err docker_err
killall dockerd
rm /var/run/docker.pid 2>/dev/null
set -x
sudo dockerd -b none --iptables=0 -l warn

In [ ]:
!sudo docker pull container-registry.oracle.com/database/express:21.3.0-xe

21.3.0-xe: Pulling from database/express

63183c9b4598: Pulling fs layer 
fe297bb4960b: Pulling fs layer 
31dd4858f370: Pulling fs layer 

Digest: sha256:016d1a2becd9c9b9bfb683eebf3aa092527fe1354ace5b23691e75759f301bed
Status: Downloaded newer image for container-registry.oracle.com/database/express:21.3.0-xe
container-registry.oracle.com/database/express:21.3.0-xe


In [ ]:
!sudo docker images

REPOSITORY                                       TAG         IMAGE ID       CREATED        SIZE
container-registry.oracle.com/database/express   21.3.0-xe   c273dde6b184   9 months ago   11.2GB


In [ ]:
!sudo docker export $(docker create container-registry.oracle.com/database/express:21.3.0-xe) > img.tar

In [ ]:
# create img folder that we can chroot in (since docker itself does not work in google colab)
!mkdir img

In [ ]:
!tar -C img  --exclude="dev" -xf img.tar && ls -lah

total 11G
drwxr-xr-x  1 root root 4.0K Jul  5 05:13 .
drwxr-xr-x  1 root root 4.0K Jul  5 04:56 ..
drwxr-xr-x  4 root root 4.0K Jun 29 13:29 .config
drwxr-xr-x 16 root root 4.0K Jul  5 05:15 img
-rw-r--r--  1 root root  11G Jul  5 05:13 img.tar
drwxr-xr-x  1 root root 4.0K Jun 29 13:30 sample_data


In [ ]:
%%bash
cd img
sudo bash
rm -Rf proc 2>/dev/null
rm -Rf sys 2>/dev/null
rm -Rf dev 2>/dev/null
mkdir proc
mkdir sys
mkdir dev
sudo mount -t proc /proc proc/
sudo mount --rbind /sys sys/
sudo mount --rbind /dev dev/

In [ ]:
%killbgscripts

All background processes were killed.


# Run oracle express

In [ ]:
!kill $(ps -A|grep ora|grep -oP "^..[0-9]+"|grep -oP "[0-9]+")

In [ ]:
!ps -A|grep ora

In [ ]:
!killall tnslsnr

In [ ]:
%%writefile start_db.sh
#!/bin/bash
#%%script bash --bg --out script_out --err script_err
#%%script bash
kill $(ps -A|grep ora|grep -oP "^..[0-9]+"|grep -oP "[0-9]+")
sudo bash
sudo chroot img /bin/bash sDB.sh

Overwriting start_db.sh


In [ ]:
%%writefile img/sDB.sh
export ORACLE_BASE=/opt/oracle
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export ORACLE_BASE_HOME=/opt/oracle/homes/OraDBHome21cXE
export ORACLE_PWD=pwd123456

export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
export PWD_FILE="setPassword.sh"
export RUN_FILE="runOracle.sh"
export START_FILE="startDB.sh"
export CREATE_DB_FILE="createDB.sh"
export SETUP_LINUX_FILE="setupLinuxEnv.sh"
export CHECK_SPACE_FILE="checkSpace.sh"
export CHECK_DB_FILE="checkDBStatus.sh"
export USER_SCRIPTS_FILE="runUserScripts.sh"
export INSTALL_DB_BINARIES_FILE="installDBBinaries.sh"
export INSTALL_RSP="db_inst.rsp"
export CONFIG_RSP="dbca.rsp.tmpl"
export CONF_FILE="oracle-xe-21c.conf"

killall tnslsnr

cd /opt/oracle/

$ORACLE_HOME/root.sh
chmod +x ./$RUN_FILE
su oracle
echo|./$RUN_FILE

Writing img/sDB.sh


In [ ]:
!chmod +x start_db.sh && echo|bash ./start_db.sh

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
tnslsnr: no process found
Check /opt/oracle/product/21c/dbhomeXE/install/root_0df340d835e9_2023-07-05_06-35-43-539377490.log for the output of root script
Specify a password to be used for database accounts. Oracle recommends that the password entered should be at least 8 characters in length, contain at least 1 uppercase character, 1 lower case character and 1 digit [0-9]. Note that the same password will be used for SYS, SYSTEM and PDBADMIN accounts:
Confirm the password:
Configuring Oracle Listener.
Listener configuration succeeded.
Configuring Oracle Database XE.
Enter SYS user password: 
********* **
Enter SYSTEM user password: 
********* ****
Enter PDBADMIN User Password: 
********* 
Prepare for db operation
7% complete
Copying database files
29% complete
Creating and starting Oracle instance
30% complete
33% complete
37% complete
40% complete
43% complete
Completing Database Creati

# Prepare script files

In [ ]:
%%writefile /content/img/script.sql
SELECT uid FROM dual;

Writing /content/img/script.sql


In [ ]:
%%writefile /content/img/add_user.sql
CREATE USER john IDENTIFIED BY pwd123456;
GRANT CREATE SESSION TO john;
GRANT CREATE ANY PROCEDURE TO john;
GRANT CREATE ANY TABLE TO john;
GRANT CREATE ANY VIEW TO john;

Writing /content/img/add_user.sql


In [ ]:
%%writefile /content/img/add_packages.sql
CREATE OR REPLACE PACKAGE test AS
  PROCEDURE test_proc;
END test;
/

CREATE OR REPLACE PACKAGE BODY test AS
  PROCEDURE test_proc AS
  BEGIN
    NULL;
  END;
END test;
/
quit

Writing /content/img/add_packages.sql


In [ ]:
%%writefile /content/img/add_tables.sql
CREATE TABLE TEST_TABLE
(
  test_col VARCHAR(255)
);

quit

Writing /content/img/add_tables.sql


In [ ]:
%%writefile /content/img/add_views.sql
CREATE VIEW TEST_VIEW AS
  SELECT * FROM TEST_TABLE;

quit

Writing /content/img/add_views.sql


In [ ]:
%%writefile /content/img/select_objects.sql

set colsep ,     -- separate columns with a comma
set pagesize 0   -- No header rows
set trimspool on -- remove trailing blanks
set headsep off  -- this may or may not be useful...depends on your headings.
set linesize 4096   -- X should be the sum of the column widths
set numw 4096       -- X should be the length you want for numbers (avoid scientific notation on IDs)

spool select.csv
SELECT * FROM all_objects WHERE LOWER(OWNER)='john';
spool off

Writing /content/img/select_objects.sql


In [ ]:
!sudo ps -AF|grep createDB|grep -v root

54321      33836   33812  0  3254  2556   1 06:21 ?        00:00:00 /bin/bash /opt/oracle/createDB.sh XE XEPDB1 pwd123456


In [ ]:
!sudo apt-get -y install net-tools && netstat -tulpen

Reading package lists... Done
Building dependency tree       
Reading state information... Done
net-tools is already the newest version (1.60+git20180626.aebd88e-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Active Internet connections (only servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       User       Inode      PID/Program name    
tcp        0      0 0.0.0.0:41415           0.0.0.0:*               LISTEN      54321      2625243    35105/xe_d000_XE    
tcp        0      0 0.0.0.0:1521            0.0.0.0:*               LISTEN      54321      2577954    33920/tnslsnr       
tcp        0      0 172.28.0.12:6000        0.0.0.0:*               LISTEN      0          19750      33/kernel_manager_p 
tcp        0      0 127.0.0.1:44559         0.0.0.0:*               LISTEN      0          2309100    25959/python3       
tcp        0      0 127.0.0.1:3453          0.0.0.0:*               LISTEN      0          19976      62/

# Execute script files

In [ ]:
%%bash
sudo chroot img
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
sqlplus sys/pwd123456@//127.0.0.1:1521/XE AS SYSDBA @script.sql


SQL*Plus: Release 21.0.0.0.0 - Production on Wed Jul 5 06:48:11 2023
Version 21.3.0.0.0

Copyright (c) 1982, 2021, Oracle.  All rights reserved.


Connected to:
Oracle Database 21c Express Edition Release 21.0.0.0.0 - Production
Version 21.3.0.0.0


       UID
----------
	 0

SQL> Disconnected from Oracle Database 21c Express Edition Release 21.0.0.0.0 - Production
Version 21.3.0.0.0


bash: no job control in this shell
bash-4.2# export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
bash-4.2# export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
bash-4.2# export ORACLE_SID=XE
bash-4.2# sqlplus sys/pwd123456@//127.0.0.1:1521/XE AS SYSDBA @script.sql
bash-4.2# exit


In [ ]:
%%bash
sudo chroot img
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
sqlplus sys/pwd123456@//localhost:1521/XEPDB1 AS SYSDBA @add_user.sql


SQL*Plus: Release 21.0.0.0.0 - Production on Wed Jul 5 06:48:15 2023
Version 21.3.0.0.0

Copyright (c) 1982, 2021, Oracle.  All rights reserved.


Connected to:
Oracle Database 21c Express Edition Release 21.0.0.0.0 - Production
Version 21.3.0.0.0


User created.


Grant succeeded.


Grant succeeded.


Grant succeeded.


Grant succeeded.

SQL> Disconnected from Oracle Database 21c Express Edition Release 21.0.0.0.0 - Production
Version 21.3.0.0.0


bash: no job control in this shell
bash-4.2# export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
bash-4.2# export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
bash-4.2# export ORACLE_SID=XE
bash-4.2# sqlplus sys/pwd123456@//localhost:1521/XEPDB1 AS SYSDBA @add_user.sql
bash-4.2# exit


In [ ]:
%%bash
chroot img
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_packages.sql
sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_tables.sql
sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_views.sql


SQL*Plus: Release 21.0.0.0.0 - Production on Wed Jul 5 06:48:20 2023
Version 21.3.0.0.0

Copyright (c) 1982, 2021, Oracle.  All rights reserved.


Connected to:
Oracle Database 21c Express Edition Release 21.0.0.0.0 - Production
Version 21.3.0.0.0


Package created.


Package body created.

Disconnected from Oracle Database 21c Express Edition Release 21.0.0.0.0 - Production
Version 21.3.0.0.0

SQL*Plus: Release 21.0.0.0.0 - Production on Wed Jul 5 06:48:21 2023
Version 21.3.0.0.0

Copyright (c) 1982, 2021, Oracle.  All rights reserved.

Last Successful login time: Wed Jul 05 2023 06:48:21 +00:00

Connected to:
Oracle Database 21c Express Edition Release 21.0.0.0.0 - Production
Version 21.3.0.0.0


Table created.

Disconnected from Oracle Database 21c Express Edition Release 21.0.0.0.0 - Production
Version 21.3.0.0.0

SQL*Plus: Release 21.0.0.0.0 - Production on Wed Jul 5 06:48:21 2023
Version 21.3.0.0.0

Copyright (c) 1982, 2021, Oracle.  All rights reserved.

Last Successful login t

bash: no job control in this shell
bash-4.2# export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
bash-4.2# export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
bash-4.2# export ORACLE_SID=XE
bash-4.2# sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_packages.sql
bash-4.2# sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_tables.sql
bash-4.2# sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_views.sql
bash-4.2# exit


In [ ]:
%%bash
chroot img
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
sqlplus john/pwd123456@//localhost:1521/XEPDB1 @select_objects.sql 1> select_1.txt 2> select_2.txt

bash: no job control in this shell
bash-4.2# export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
bash-4.2# export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
bash-4.2# export ORACLE_SID=XE
select_1.txt 2> select_2.txt
bash-4.2# exit


# Get source of git-remote-oracle

In [ ]:
%%bash
git clone https://github.com/TheCrazyT/git-remote-oracle

Cloning into 'git-remote-oracle'...


In [ ]:
%%bash
cd git-remote-oracle
git checkout git-remote-oracle
git status
git pull

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Already up to date.


error: pathspec 'git-remote-oracle' did not match any file(s) known to git


# Install dependencies

In [ ]:
!pip install oracledb pexpect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 39.0 MB/s eta 0:00:00


# Test git clone

In [ ]:
%%bash
export PATH=$PATH:$(pwd)/git-remote-oracle/src/git_remote_oracle
export GR_ORACLE_DEBUG=1
export GR_ORACLE_DEBUG_GIT=1

export REL_PATH=$(pwd)/git-remote-oracle/src/git_remote_oracle

echo "#!/bin/bash" > $(pwd)/git-remote-oracle/src/git_remote_oracle/git-remote-oracle
echo python3 $(pwd)'/git-remote-oracle/src/git_remote_oracle/git_remote_oracle.py $@' >> $(pwd)/git-remote-oracle/src/git_remote_oracle/git-remote-oracle

chmod +x ${REL_PATH}/git_remote_oracle.py
chmod +x ${REL_PATH}/git-remote-oracle

rm -Rf JOHN 2>/dev/null

git config --global credential.helper store
echo -e "protocol=oracle\nhost=127.0.0.1\nusername=john\npassword=pwd123456"|git credential approve
git clone oracle://127.0.0.1:1521/XEPDB1/JOHN

Cloning into 'JOHN'...
START
['/content/git-remote-oracle/src/git_remote_oracle/git_remote_oracle.py', 'origin', 'oracle://127.0.0.1:1521/XEPDB1/JOHN']
oracle://127.0.0.1:1521/XEPDB1/JOHN
host_and_path: 127.0.0.1:1521/XEPDB1/JOHN
host_and_port: 127.0.0.1:1521
host: 127.0.0.1
port: 1521
service_and_schema: XEPDB1/JOHN
service: XEPDB1
schema: JOHN
expected 'Username for' End Of File (EOF). Exception style platform.
command: /usr/lib/git-core/git
args: ['/usr/lib/git-core/git', 'credential', 'fill']
buffer (last 100 chars): b''
before (last 100 chars): b'protocol=oracle\r\nhost=127.0.0.1\r\nusername=john\r\npassword=pwd123456\r\n'
after: <class 'pexpect.exceptions.EOF'>
match: None
match_index: None
exitstatus: 0
flag_eof: True
pid: 44099
child_fd: 5
closed: False
timeout: 30
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfile_read: None
logfile_send: None
maxread: 2000
ignorecase: False
searchwindowsize: None
delaybeforesend: 0.05
delayafterclose: 0.1
delayafterterminate: 0

# Test git pull

In [ ]:
%%bash
export PATH=$PATH:$(pwd)/git-remote-oracle/src/git_remote_oracle
export GR_ORACLE_DEBUG=1
export GR_ORACLE_DEBUG_GIT=1

cd JOHN
git pull

Already up to date.


START
['/content/git-remote-oracle/src/git_remote_oracle/git_remote_oracle.py', 'origin', 'oracle://127.0.0.1:1521/XEPDB1/JOHN']
oracle://127.0.0.1:1521/XEPDB1/JOHN
host_and_path: 127.0.0.1:1521/XEPDB1/JOHN
host_and_port: 127.0.0.1:1521
host: 127.0.0.1
port: 1521
service_and_schema: XEPDB1/JOHN
service: XEPDB1
schema: JOHN
expected 'Username for' End Of File (EOF). Exception style platform.
command: /usr/lib/git-core/git
args: ['/usr/lib/git-core/git', 'credential', 'fill']
buffer (last 100 chars): b''
before (last 100 chars): b'protocol=oracle\r\nhost=127.0.0.1\r\nusername=john\r\npassword=pwd123456\r\n'
after: <class 'pexpect.exceptions.EOF'>
match: None
match_index: None
exitstatus: 0
flag_eof: True
pid: 44259
child_fd: 5
closed: False
timeout: 30
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfile_read: None
logfile_send: None
maxread: 2000
ignorecase: False
searchwindowsize: None
delaybeforesend: 0.05
delayafterclose: 0.1
delayafterterminate: 0.1
searcher: searcher_r

# List files

In [ ]:
!ls -lah JOHN
!ls -lah JOHN/PACKAGE

total 32K
drwxr-xr-x 7 root root 4.0K Jul  5 06:49  .
drwxr-xr-x 1 root root 4.0K Jul  5 06:49  ..
drwxr-xr-x 8 root root 4.0K Jul  5 06:49  .git
-rw-r--r-- 1 root root   19 Jul  5 06:49  last_ddl
drwxr-xr-x 2 root root 4.0K Jul  5 06:49  PACKAGE
drwxr-xr-x 2 root root 4.0K Jul  5 06:49 'PACKAGE BODY'
drwxr-xr-x 2 root root 4.0K Jul  5 06:49  TABLE
drwxr-xr-x 2 root root 4.0K Jul  5 06:49  VIEW
total 12K
drwxr-xr-x 2 root root 4.0K Jul  5 06:49 .
drwxr-xr-x 7 root root 4.0K Jul  5 06:49 ..
-rw-r--r-- 1 root root   90 Jul  5 06:49 TEST.sql


# Stop oracle

In [ ]:
%killbgscripts

All background processes were killed.
